<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1


    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-06 14:17:56
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.35 C
-------------------
Today PnL: -1.02 L (-0.75%)
Current PnL: -19.89 L (-13.53%)
CY Booked + Current PnL: -11.62 L (-7.91%)
-------------------
Total profit:  2.18 L
Total loss:  -22.07 L
-------------------
Total Booked + Current PnL: 15.36 L (12.79%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.13%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 78.16 L (57.91%)
Deployed:  1.20 C
Current:  1.35 C
CAGR/XIRR %: 7.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,761.55,-19.90,H-LC,5.30,110048.0,5736.0,11797.0,0.10,5.50,10.72,16.81,10.0,0.49,0.84,22.43,X5K,ATH,METALS
67,SBIN,760.30,863.00,-11.51,M-LC,3.97,213431.0,11952.0,15260.0,0.60,5.93,7.15,13.51,62.0,0.78,1.62,19.28,XY25,NTT,BANKS
27,DIXON,14367.00,18940.15,-26.82,H-MC,6.57,134792.0,19856.0,16728.0,-0.90,17.28,12.41,31.83,85.0,1.19,1.03,51.70,X40N,ATH,IT
82,TTKPRESTIG,769.29,770.00,-9.13,H-SC,11.64,82988.0,-17789.0,17884.0,0.78,-17.65,21.55,0.09,145.0,-0.99,0.63,9.03,OX40N,NTT,DURABLES
71,SIS,477.00,477.00,2239.90,H-SC,7.00,62951.0,-18139.0,18136.0,-1.03,-22.37,28.81,0.00,157.0,-1.00,0.48,27.58,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,NATIONALUM,189.63,247.44,-50.15,H-MC,2.87,99889.0,-615.0,31255.0,0.65,-0.61,31.29,30.49,81.0,-0.02,0.76,31.59,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.84,H-LC,1.79,202870.0,-1630.0,43049.0,-1.27,-0.80,21.22,20.25,9.0,-0.04,1.54,7.87,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-16.63,H-LC,2.82,165702.0,-35684.0,123299.0,-0.53,-17.72,74.41,43.51,19.0,-0.29,1.26,15.35,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-5.66,H-LC,3.24,181638.0,-18672.0,78649.0,-1.68,-9.32,43.30,29.94,24.0,-0.24,1.38,25.48,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.42,H-LC,3.75,185943.0,-14563.0,63332.0,-0.74,-7.26,34.06,24.33,30.0,-0.23,1.41,7.23,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-17.42,H-LC,4.19,261741.0,-3685.0,60593.0,-1.33,-1.39,23.15,21.44,26.0,-0.06,1.99,5.16,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UNIONBANK,123.87,163.00,-13.71,M-LC,15.61,149231.0,8391.0,36099.0,2.55,5.96,24.19,31.59,72.0,0.23,1.14,34.02,XY24,NTT,BANKS
2,ACC,2579.05,3906.00,-38.07,M-MC,4.88,133583.0,-54688.0,151550.0,1.56,-29.05,113.45,51.45,178.0,-0.36,1.02,2.32,XY24,BTT,CEMENT
54,MASFIN,326.60,399.50,-19.15,H-SC,16.35,91905.0,-6075.0,27948.0,1.42,-6.20,30.41,22.32,151.0,-0.22,0.70,33.29,XR,ATH,FINANCE
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.40,133534.0,10198.0,123572.0,1.32,8.27,92.54,108.45,118.0,0.08,1.02,32.47,AR,ATH,MISC
22,COALINDIA,406.65,504.72,0.00,L-LC,2.25,139730.0,-10730.0,47019.0,0.98,-7.13,33.65,24.12,169.0,-0.23,1.06,8.19,XY25,ATH,MINING


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BRITANNIA,5502.66,6362.67,8.15,H-LC,9.29,194130.0,-3966.0,34924.0,-4.31,-2.00,17.99,15.63,31.0,-0.11,1.48,19.41,XY25,ATH,FMCG
60,RAJESHEXPO,517.65,518.00,1662.63,L-SC,3.43,48098.0,-89079.0,89174.0,-4.02,-64.94,185.40,0.07,266.0,-1.00,0.37,19.62,OX40N,NTT,JEWELLERY
75,SYMPHONY,1306.42,1306.00,-39.43,M-SC,9.43,129258.0,-41883.0,41828.0,-3.28,-24.47,32.36,-0.03,197.0,-1.00,0.98,0.00,OX40N,NTT,DURABLES
74,SURYODAY,189.78,240.00,40.35,H-SC,5.06,99366.0,-50560.0,90234.0,-3.05,-33.72,90.81,26.46,142.0,-0.56,0.76,27.29,XR,NTT,BANKS
49,KANSAINER,299.63,340.00,-69.48,H-SC,1.66,212211.0,-57456.0,93797.0,-2.63,-21.31,44.20,13.47,136.0,-0.61,1.61,7.33,XY24,NTT,PAINTS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,129.35,H-SC,4.67,150486.0,-24073.0,24108.0,0.71,-13.79,16.02,0.02,135.0,-1.00,1.14,30.49,OX40N,NTT,PAINTS
82,TTKPRESTIG,769.29,770.0,-9.13,H-SC,11.64,82988.0,-17789.0,17884.0,0.78,-17.65,21.55,0.09,145.0,-0.99,0.63,9.03,OX40N,NTT,DURABLES
83,UJJIVANSFB,52.77,53.0,34.19,H-SC,20.64,116343.0,-26136.0,26759.0,-2.05,-18.34,23.00,0.44,163.0,-0.98,0.89,37.32,OX40N,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-22.18,H-MC,2.06,81662.0,-20990.0,20987.0,-2.16,-20.45,25.70,-0.00,101.0,-1.00,0.62,24.14,OX40N,NTT,IT
20,CERA,7989.07,8422.0,-20.76,H-SC,2.84,113202.0,-30601.0,38398.0,-1.82,-21.28,33.92,5.42,155.0,-0.80,0.86,24.92,OX40N,NTT,CERAMICS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.38,H-LC,14.87,236925.0,-59009.0,152911.0,-0.86,-19.94,64.54,31.73,1.0,-0.39,1.80,1.15,X40,BTT,IT
88,VBL,492.64,671.64,-11.07,H-LC,11.34,269861.0,4821.0,91483.0,-1.28,1.82,33.90,36.33,3.0,0.05,2.05,15.36,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-19.60,H-LC,8.68,257545.0,-4056.0,149685.0,-1.47,-1.55,58.12,55.67,4.0,-0.03,1.96,4.46,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.44,H-LC,8.75,198178.0,-18341.0,111237.0,-0.05,-8.47,56.13,42.91,5.0,-0.16,1.51,3.59,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,0.53,H-LC,9.08,178547.0,-12099.0,58617.0,-1.29,-6.35,32.83,24.40,8.0,-0.21,1.36,8.58,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,20.18,H-SC,13.46,154036.0,27836.0,18346.0,0.08,22.06,11.91,36.59,156.0,1.52,1.17,37.34,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.10,H-SC,13.49,132212.0,6915.0,52158.0,-1.47,5.52,39.45,47.15,144.0,0.13,1.01,65.41,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.46,154036.0,27836.0,18346.0,0.08,22.06,11.91,36.59,156.0,1.52,1.17,37.34,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-2.38,H-MC,4.91,156849.0,5936.0,51368.0,-1.44,3.93,32.75,37.97,90.0,0.12,1.19,30.23,AR,ATH,AUTO
19,CAMS,3643.00,5226.82,-4.80,H-SC,1.77,106940.0,4936.0,39407.0,0.60,4.84,36.85,43.48,120.0,0.13,0.81,22.43,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2326.38,-6.96,H-SC,8.68,174630.0,9960.0,58012.0,-0.25,6.05,33.22,41.28,119.0,0.17,1.33,14.51,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.10,H-SC,13.49,132212.0,6915.0,52158.0,-1.47,5.52,39.45,47.15,144.0,0.13,1.01,65.41,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,-2.38,H-MC,4.91,156849.0,5936.0,51368.0,-1.44,3.93,32.75,37.97,90.0,0.12,1.19,30.23,AR,ATH,AUTO
90,WHIRLPOOL,1219.98,2270.00,-40.95,M-SC,5.70,99278.0,7780.0,70974.0,0.49,8.50,71.49,86.07,211.0,0.11,0.76,43.62,XR,NTT,DURABLES
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.40,133534.0,10198.0,123572.0,1.32,8.27,92.54,108.45,118.0,0.08,1.02,32.47,AR,ATH,MISC
91,WIPRO,243.46,420.00,-12.85,M-LC,5.90,149432.0,-1513.0,110968.0,-1.99,-1.00,74.26,72.51,55.0,-0.01,1.14,4.96,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-55.73,L-SC,21.60,66413.0,-47136.0,87187.0,-1.82,-41.51,131.28,35.27,267.0,-0.54,0.51,62.55,XR,NTT,HOTELS
54,MASFIN,326.60,399.50,-19.15,H-SC,16.35,91905.0,-6075.0,27948.0,1.42,-6.20,30.41,22.32,151.0,-0.22,0.70,33.29,XR,ATH,FINANCE
3,ALKYLAMINE,2347.98,4546.37,-0.30,H-SC,18.79,92760.0,-8203.0,102732.0,-2.56,-8.12,110.75,93.63,149.0,-0.08,0.71,38.16,SR,ATH,CHEMICALS
87,VALIANTORG,512.64,838.00,-323.33,H-SC,19.80,94874.0,-38412.0,123004.0,0.08,-28.82,129.65,63.47,147.0,-0.31,0.72,59.03,XR,NTT,CHEMICALS
36,HINDALCO,651.95,761.55,-19.90,H-LC,5.30,110048.0,5736.0,11797.0,0.10,5.50,10.72,16.81,10.0,0.49,0.84,22.43,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-23.54,H-SC,7.01,217709.0,24689.0,23360.0,0.41,12.79,10.73,24.89,128.0,1.06,1.66,39.12,X40N,NTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.46,154036.0,27836.0,18346.0,0.08,22.06,11.91,36.59,156.0,1.52,1.17,37.34,XY24,NTT,HEALTHCARE
69,SHALBY,261.39,327.00,943.06,M-SC,8.41,141788.0,-39355.0,84818.0,-1.08,-21.73,59.82,25.10,233.0,-0.46,1.08,13.63,XY24,NTT,HEALTHCARE
88,VBL,492.64,671.64,-11.07,H-LC,11.34,269861.0,4821.0,91483.0,-1.28,1.82,33.90,36.33,3.0,0.05,2.05,15.36,X40N,ATH,FMCG
27,DIXON,14367.00,18940.15,-26.82,H-MC,6.57,134792.0,19856.0,16728.0,-0.90,17.28,12.41,31.83,85.0,1.19,1.03,51.70,X40N,ATH,IT


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.69
1,25,41.46
2,50,71.37


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.69
LC    37.10
MC    22.22
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.96
X40N     15.18
X40      13.26
XY25     11.70
XR        9.01
AR        7.69
OX40N     7.68
X5K       2.23
MH        1.77
X200      1.38
SR        1.15
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.94
H-SC    27.36
H-MC    19.23
M-SC    11.90
M-LC     7.10
M-MC     2.64
L-SC     1.43
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.21,-3.08,36.22
IT,12.01,-18.15,75.54
FINANCE,8.35,-18.75,61.84
BANKS,7.52,-15.48,62.61
PAINTS,5.76,-15.03,37.70
MISC,5.12,-21.54,92.65
HEALTHCARE,5.11,2.27,22.41
ELECTRICAL,4.71,-13.70,62.95
AUTO,4.39,-17.04,67.96


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2830101.0,23
XR,1110499.0,13
AR,1054572.0,9
X40,830575.0,10
X40N,610816.0,12
XY25,510534.0,8
OX40N,367885.0,10
SR,192868.0,2
X5K,146247.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2726885.0,30
M-SC,1406489.0,15
H-LC,1349627.0,20
H-MC,1178682.0,15
M-LC,445275.0,5
M-MC,331481.0,2
L-SC,274016.0,3
L-MC,56685.0,1
L-LC,47019.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      950578.0      7
M-SC       XY24      859431.0      7
H-SC       AR        611186.0      4
H-LC       X40       553494.0      6
H-SC       XR        517745.0      6
H-MC       XY24      461971.0      4
M-MC       XY24      331481.0      2
H-LC       X40N      313460.0      6
           AR        233364.0      2
H-SC       X40N      215801.0      4
H-MC       XY25      209933.0      2
           X40       201329.0      3
H-SC       SR        192868.0      2
           OX40N     186549.0      6
L-SC       XR        184842.0      2
M-LC       XY24      184597.0      2
M-SC       AR        158654.0      2
M-LC       X5K       134450.0      1
M-SC       XY25      121502.0      1
H-MC       XR        120284.0      1
M-SC       XR        119975.0      2
H-LC       XY25      116820.0      3
M-LC       XR        110968.0      1
L-SC       OX40N      89174.0      1
H-MC       X40N       81555.0      2
H-LC       X200       78649.0      1
M-SC       X40        75752.0      1
           OX40N      71175.0      2
L-MC       XR         56685.0      1
H-SC       MH         52158.0      1
H-MC       AR         51368.0      1
L-LC       XY25       47019.0      1
H-LC       XY24       42043.0      1
H-MC       MH         31255.0      1
           OX40N      20987.0      1
M-LC       XY25       15260.0      1
H-LC       X5K        11797.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
